In [1]:
import scipy.io as sciio
import numpy as np
import re
import matplotlib.pyplot as plt
import scipy.interpolate as sp_int

def parse_stl(filename):
    match_number = "-{0,1}[0-9e\+\.-]+"
    parse_normal = re.compile("^\s*facet normal(\s+{0})(\s+{0})(\s+{0})".format(match_number))
    parse_vertex = re.compile("^\s*vertex(\s+{0})(\s+{0})(\s+{0})".format(match_number))
    parse_endfacet = re.compile("^endfacet")

    with open(filename, 'r') as stl_file:
        lines = stl_file.readlines()
        num_elements = (len(lines)-2)//7
        stl_content = np.zeros((num_elements, 4, 3))
        element_id = 0
        vertex_id = 1
        for l in lines:
            if parse_normal.match(l):
                normal = np.array([float(f) for f in parse_normal.findall(l)[0]])
                stl_content[element_id, 0, :] = normal
            if parse_vertex.match(l):
                coord = np.array([float(f) for f in parse_vertex.findall(l)[0]])
                stl_content[element_id, vertex_id, :] = coord
                vertex_id += 1
            if parse_endfacet.match(l):
                vertex_id = 1
                element_id += 1
        return stl_content

def write_stl(filename, stl):
    with open(filename, 'w+') as stl_file:
        stl_file.write('solid Created by python\n')
        for element in range(stl.shape[0]):
            stl_file.write('facet normal {} {} {}\n'.format(stl[element, 0, 0],stl[element, 0, 1],stl[element, 0, 2]))
            stl_file.write('  outer loop\n')
            stl_file.write('    vertex {} {} {}\n'.format(stl[element, 1, 0],stl[element, 1, 1],stl[element, 1, 2]))
            stl_file.write('    vertex {} {} {}\n'.format(stl[element, 2, 0],stl[element, 2, 1],stl[element, 2, 2]))
            stl_file.write('    vertex {} {} {}\n'.format(stl[element, 3, 0],stl[element, 3, 1],stl[element, 3, 2]))
            stl_file.write('  endloop\n')
            stl_file.write('endfacet\n')
        stl_file.write('endsolid Created by python\n')

def interpolate_stl(stl, x_0, y_0, z_0):
    stl_interpolated = np.zeros(stl.shape)
    interp = sp_int.interp2d(x_0, y_0, z_0)

    stl_interpolated[:, :, 1] = stl[:, :, 1]
    stl_interpolated[:, :, 0] = stl[:, :, 0]
    for element in range(stl.shape[0]):
        #interpolate z axis
        for vertex in range(1,4):
            stl_interpolated[element, vertex, 2] = interp(stl[element, vertex, 0], stl[element, vertex, 1])
        #fix normal
        stl_interpolated[element, 0, :] = np.cross(stl_interpolated[element, 2, :] - stl_interpolated[element, 1, :], 
        stl_interpolated[element, 3, :] - stl_interpolated[element, 1, :])
        stl_interpolated[element, 0, :] = stl_interpolated[element, 0, :] / np.linalg.norm(stl_interpolated[element, 0, :])

    return stl_interpolated


In [2]:
path_to_dem = "../data/Kilauea_DEM_and_lavalakelocation.mat"

In [3]:
dem = sciio.loadmat(path_to_dem)['DEM']
x_lake= 2.604584249773606e+05
y_lake= 2.147270125870880e+06

xs = dem['x'][0,0] - x_lake
ys = dem['y'][0,0] - y_lake
zs = dem['Z'][0,0]

print(xs.min(), xs.max())
print(ys.min(), ys.max())


skip_nth = 10
xs = xs[::skip_nth,:]
ys = ys[::skip_nth,:]
zs = zs[::skip_nth,::skip_nth]

xs.shape, zs.shape

-9983.482789544796 15443.977492558566
-12155.990126644727 10576.767193291802


((252, 1), (225, 252))

In [6]:
path_to_crater = "../data/Contour1100.mat"
crater = sciio.loadmat(path_to_crater)
crater_x = crater['x'][:,0] - x_lake
crater_y = crater['y'][:,0] - y_lake


from scipy.spatial import ConvexHull

crater_points = np.vstack((crater_x, crater_y)).T
hull = ConvexHull(crater_points)


def point_in_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

for i in range(xs.shape[0]):
    for j in range(ys.shape[0]):
        if point_in_hull(np.array([xs[i], ys[j]]), hull):
            zs[i,j] = 10000000 #1030
       # zs[i,j] = (1100, zs[i,j])

In [ ]:
plt.scatter(crater_x, crater_y)
plt.plot(crater_points[hull.vertices,0], crater_points[hull.vertices,1], 'r--', lw=2)

In [ ]:
X, Y = np.meshgrid(crater_x, y)
plt.contourf(X, Y, zs, levels=100)
plt.xlim(-2e3,2e3)
plt.ylim(-2e3, 2e3)
plt.show()

In [ ]:
#in_hull = np.all(np.add(np.dot(random_points, hull.equations[:,:-1].T),
#hull.equations[:,-1]) <= tolerance, axis=1)


stl_plane = parse_stl('../meshes/topography_plane.stl')
stl_interpolated = interpolate_stl(stl_plane, xs, ys, zs)
write_stl('../meshes/topography.stl', stl_interpolated)

In [ ]:
zs.min()

In [ ]:
crater_x.mean()

In [ ]:
crater['x'][:,0].max()